<h1>Simulation<h1>

In [1]:
from vpython import * 
scene = canvas()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<h2>Parameters<h2>

In [2]:
k = 1
m = 1
distance = 1
r = .3
Length_spring = 1 - 1.8*.3
Volume_spring = pi*(.15)**2*Length_spring
dt = .1
t = 0
U = 0
KE = 0
spring_max = 0
v = 0
dx = Length_spring

In [3]:
g1 = graph(xtitle="t [s]", ytitle="Energy", title="Euler Forward", width=500, height=150)
f1 = gcurve(color=color.red)
f2 = gcurve(color=color.green)
f3 = gcurve(color=color.blue)

g2 = graph(xtitle="t [s]", ytitle="Energy", title="Euler Backward",width=500, height=150)
f4 = gcurve(color=color.red)
f5 = gcurve(color=color.green)
f6 = gcurve(color=color.blue)

g3 = graph(xtitle="t [s]", ytitle="Energy", title="Trapezoid", width=500, height=150)
f7 = gcurve(color=color.red)
f8 = gcurve(color=color.green)
f9 = gcurve(color=color.blue)

g4 = graph(xtitle="t [s]", ytitle="Energy", title="Damped", width=500, height=150)
f10 = gcurve(color=color.red)
f11 = gcurve(color=color.green)
f12 = gcurve(color=color.blue)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<h2>Spring Function<h2>

<h4>Background Knowledge<h4>

In my previous trail of this project, I was able to create the matrix, however it did not perform the intended motion. This is because of my misunderstand on how the structure would function as a physical system. I had initially made a 3x3x3 cube of 27 atoms and springs connecting all the adjacents. After doing research and studying the example simulation further, I realized that I this is not similar to how this structure interacts with the real world at all, but rather that all atos will have an adjacent atom. Because of this I need to create a few more layers of atoms that are not necessarily visible, in order to apply the correct forces  to the atoms I intend to simulate, otherwise the atoms moment will build and begin moving across the screen in a big blob of a figure rather than a cube.

In [4]:
class Crystal:
        
    def __init__(self, radius, spacing, momentumRange):
        self.atoms = []
        self.springs = []
        
        for z in range(-1,4,1):
            for y in range(-1,4,1):
                for x in range(-1,4,1):
                    atom = sphere(pos=vector(x,y,z)*spacing, radius = radius, color = color.blue)
                    #only apply velocity to the visible atoms, the remaining atoms will keep the moving atoms in the intended place
                    if 0 <= x < 3 and 0 <= y < 3 and 0 <= z < 3:
                        p = vec.random()
                        atom.momentum = momentumRange*p
                        #for m = 1, p = v
                        atom.velocity = atom.momentum/m
                    else:#outer most layer is invisible
                        atom.visible = False
                        atom.velocity = vec(0,0,0)
                        atom.momentum = vec(0,0,0)
                    atom.index = len(self.atoms)
                    self.atoms.append(atom)
                    
        for atom in self.atoms:
            if atom.visible:
                if atom.pos.x == 0 or atom.pos.x == 2:
                    self.place_spring("x", atom.pos.x, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+1], True)

                if atom.pos.y == 0 or atom.pos.y == 2:
                    self.place_spring("y", atom.pos.y, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)], True)
                    
                if atom.pos.z == 0 or atom.pos.z == 2:
                    self.place_spring("z", atom.pos.z, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
    
    # Create a grid of springs linking each atom to the adjacent atoms
    # in each dimension, or to invisible motionless atoms
    def make_spring(self, start, end, visible):
        spring = helix(pos = start.pos, axis = end.pos-start.pos, visible = visible, points = [start, end], color = color.white)
        #spring characteristics for modeling
        spring.thickness = 0.05
        spring.radius = 0.15
        spring.length = distance
        self.springs.append(spring)
        
    def place_spring(self, c, n, atom):
        if(c=="x" and n==0):
            self.make_spring(self.atoms[atom.index-1], atom, False)
            self.make_spring(atom, self.atoms[atom.index+1], True)
        if(c=="x" and n==2):
            self.make_spring(atom, self.atoms[atom.index+1], False)
        if(c=="y" and n==0):
            self.make_spring(self.atoms[atom.index-(5)], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)], True)
        if(c=="y" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)], False)
        if(c=="z" and n==0):
            self.make_spring(self.atoms[atom.index-(5)**2], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
        if(c=="z" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)**2], False)
        


The moving atoms are only the visible ones, the invisible ones serve the purpose of keeping the moving atoms in the proper place. This simulates the real world rather well (in an ideal reality) because the net force applied to most objects by atoms is close to 0, due to the sheer quantity of the atoms and their spacing in our universe, all constantly applying forces to other atoms from every direction possible.

I would also like to note there were not many adjustments or optimizations I could figure out for the make spring loop in particular. Trying to make it in another format seems to constantly result in the atoms not moving in the correct mannerism and I am uncertain as to why. I believe it is a misconception with the invisible springs and how I was setting them up, however without any visual aid to them it is difficult to test for, and when they all do become visible the lag increases dramatically and because of this it is too difficult to tell what is actually happening.

In [5]:
lattice = Crystal(r, distance, 0.1*distance*sqrt(k/m))

<h2>Loop<h2>

<h4>Forward Euler<h4>

In [6]:
while t<10:
    rate(20)
    f1.plot(t, U)
    f2.plot(t, KE)
    f3.plot(t, U+KE)
    for atom in lattice.atoms:
        if atom.visible:
            atom.pos = atom.pos + (atom.velocity)*dt
    for spring in lattice.springs:
        spring.axis = spring.points[1].pos - spring.points[0].pos
        L = mag(spring.axis)
        spring.axis = spring.axis.norm()
        spring.pos = spring.points[0].pos+0.15*spring.axis
        spring.length = L-atom.radius
        Force = spring.axis * (k*dt * (1-distance/L))
        
        if spring.points[0].visible:
            spring.points[0].momentum += Force
            spring.points[0].velocity = spring.points[0].momentum/m
        if spring.points[1].visible:
            spring.points[1].momentum -=  Force
            spring.points[1].velocity = spring.points[1].momentum/m
            
    
    v+=(-k)*dx*dt
    dx+=v*dt
    t+=dt 
    KE=.5*m*v**2
    U=.5*k*dx**2

These results align well with my previous observations. Due to the constant movement in every direction, the net force and therefore net KE is close to 0, because so much of that energy is being oppressed by the corresponding energy. Because of this the Potential energy will be much higher, because if the particle were to free itself from these constraints, the collective net force would all go into one unopposed direction, turning into KE and continuing until another force stopped it.

𝑥 𝑡 + Δ𝑡 = 𝑥 𝑡 + 𝑣(𝑡) Δ𝑡 (Explicit / Forward Euler)
𝑥 𝑡 + Δ𝑡 = 𝑥 𝑡 + 𝒗 𝒕 + 𝚫𝒕 ∗ Δ𝑡 (Implicit / Backward Euler)
𝑥 𝑡 + Δ𝑡 = 𝑥 𝑡 + 0.5 ∗ 𝑣 𝑡 + 𝒗 𝒕 + 𝜟𝒕 ∗ Δ𝑡 (Trapezoid)

<h4>Backward Euler<h4>

In [7]:
from vpython import * 
scene = canvas()

<IPython.core.display.Javascript object>

In [8]:
k = 1
m = 1
distance = 1
r = .3
Length_spring = 1 - 1.8*.3
Volume_spring = pi*(.15)**2*Length_spring
dt = .1
t = .1
U = 0
KE = 0
spring_max = 0
v = 0
dx = Length_spring

In [9]:
class Crystal:
        
    def __init__(self, radius, spacing, momentumRange):
        self.atoms = []
        self.springs = []
        
        for z in range(-1,4,1):
            for y in range(-1,4,1):
                for x in range(-1,4,1):
                    atom = sphere(pos=vector(x,y,z)*spacing, radius = radius, color = color.blue)
                    #only apply velocity to the visible atoms, the remaining atoms will keep the moving atoms in the intended place
                    if 0 <= x < 3 and 0 <= y < 3 and 0 <= z < 3:
                        p = vec.random()
                        atom.momentum = momentumRange*p
                        #for m = 1, p = v
                        atom.velocity = atom.momentum/m
                    else:#outer most layer is invisible
                        atom.visible = False
                        atom.velocity = vec(0,0,0)
                        atom.momentum = vec(0,0,0)
                    atom.index = len(self.atoms)
                    self.atoms.append(atom)
                    
        for atom in self.atoms:
            #determine if the adjacent atom should be invisible or visible
            if atom.visible:
                if atom.pos.x == 0 or atom.pos.x == 2:
                    self.place_spring("x", atom.pos.x, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+1], True)

                if atom.pos.y == 0 or atom.pos.y == 2:
                    self.place_spring("y", atom.pos.y, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)], True)
                    
                if atom.pos.z == 0 or atom.pos.z == 2:
                    self.place_spring("z", atom.pos.z, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
    
    # Create a grid of springs linking each atom to the adjacent atoms
    # in each dimension, or to invisible motionless atoms
    def make_spring(self, start, end, visible):
        spring = helix(pos = start.pos, axis = end.pos-start.pos, visible = visible, points = [start, end], color = color.white)
        #spring characteristics for modeling
        spring.thickness = 0.05
        spring.radius = 0.15
        spring.length = distance
        self.springs.append(spring)
        
    def place_spring(self, c, n, atom):
        if(c=="x" and n==0):
            self.make_spring(self.atoms[atom.index-1], atom, False)
            self.make_spring(atom, self.atoms[atom.index+1], True)
        if(c=="x" and n==2):
            self.make_spring(atom, self.atoms[atom.index+1], False)
        if(c=="y" and n==0):
            self.make_spring(self.atoms[atom.index-(5)], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)], True)
        if(c=="y" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)], False)
        if(c=="z" and n==0):
            self.make_spring(self.atoms[atom.index-(5)**2], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
        if(c=="z" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)**2], False)
        


In [10]:
lattice2 = Crystal(r, distance, 0.1*distance*sqrt(k/m))

In [11]:
while t<10:
    rate(20)
    f4.plot(t, U)
    f5.plot(t, KE)
    f6.plot(t, U+KE)
    for atom in lattice2.atoms:
        if atom.visible:
            atom.pos = atom.pos + ((atom.velocity)+atom.velocity*dt/t)*dt
    for spring in lattice2.springs:
        spring.axis = spring.points[1].pos - spring.points[0].pos
        L = mag(spring.axis)
        spring.axis = spring.axis.norm()
        spring.pos = spring.points[0].pos+0.15*spring.axis
        spring.length = L-atom.radius
        Force = spring.axis * (k*dt * (1-distance/L))
        
        if spring.points[0].visible:
            spring.points[0].momentum += Force
            spring.points[0].velocity = spring.points[0].momentum/m
        if spring.points[1].visible:
            spring.points[1].momentum -=  Force
            spring.points[1].velocity = spring.points[1].momentum/m
            
    
    v+=(-k)*dx*dt
    dx+=v*(t+dt)*dt
    t+=dt 
    KE=.5*m*v**2
    U=.5*k*dx**2

Using the Backwards Euler formula, the Energy plot yields different results than seen in forward. Although when initially researching the topic it seemed as the Backwards Euler would yield better results for larger time steps than Forward, I found that smaller time steps seem to make the curve smoother and yields clearer results. Trying a time step of dt=1 results in no graph being able to be plotted, and a time step of dt=.5 results in a similar curve with less points, forming a group of very jagged points. Total Energy in this case also seems to constantly increase, even though the movement of the cube does not strike me as all that different.

<h2>Trapezoid<h2>

In [12]:
from vpython import * 
scene = canvas()

<IPython.core.display.Javascript object>

In [13]:
k = 1
m = 1
distance = 1
r = .3
Length_spring = 1 - 1.8*.3
Volume_spring = pi*(.15)**2*Length_spring
dt = .1
t = .1
U = 0
KE = 0
spring_max = 0
v = 0
dx = Length_spring

In [14]:
class Crystal:
        
    def __init__(self, radius, spacing, momentumRange):
        self.atoms = []
        self.springs = []
        
        for z in range(-1,4,1):
            for y in range(-1,4,1):
                for x in range(-1,4,1):
                    atom = sphere(pos=vector(x,y,z)*spacing, radius = radius, color = color.blue)
                    #only apply velocity to the visible atoms, the remaining atoms will keep the moving atoms in the intended place
                    if 0 <= x < 3 and 0 <= y < 3 and 0 <= z < 3:
                        p = vec.random()
                        atom.momentum = momentumRange*p
                        #for m = 1, p = v
                        atom.velocity = atom.momentum/m
                    else:#outer most layer is invisible
                        atom.visible = False
                        atom.velocity = vec(0,0,0)
                        atom.momentum = vec(0,0,0)
                    atom.index = len(self.atoms)
                    self.atoms.append(atom)
                    
        for atom in self.atoms:
            #determine if the adjacent atom should be invisible or visible
            if atom.visible:
                if atom.pos.x == 0 or atom.pos.x == 2:
                    self.place_spring("x", atom.pos.x, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+1], True)

                if atom.pos.y == 0 or atom.pos.y == 2:
                    self.place_spring("y", atom.pos.y, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)], True)
                    
                if atom.pos.z == 0 or atom.pos.z == 2:
                    self.place_spring("z", atom.pos.z, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
    
    # Create a grid of springs linking each atom to the adjacent atoms
    # in each dimension, or to invisible motionless atoms
    def make_spring(self, start, end, visible):
        spring = helix(pos = start.pos, axis = end.pos-start.pos, visible = visible, points = [start, end], color = color.white)
        #spring characteristics for modeling
        spring.thickness = 0.05
        spring.radius = 0.15
        spring.length = distance
        self.springs.append(spring)
        
    def place_spring(self, c, n, atom):
        if(c=="x" and n==0):
            self.make_spring(self.atoms[atom.index-1], atom, False)
            self.make_spring(atom, self.atoms[atom.index+1], True)
        if(c=="x" and n==2):
            self.make_spring(atom, self.atoms[atom.index+1], False)
        if(c=="y" and n==0):
            self.make_spring(self.atoms[atom.index-(5)], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)], True)
        if(c=="y" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)], False)
        if(c=="z" and n==0):
            self.make_spring(self.atoms[atom.index-(5)**2], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
        if(c=="z" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)**2], False)
        


In [15]:
lattice2 = Crystal(r, distance, 0.1*distance*sqrt(k/m))

In [16]:
while t<10:
    rate(20)
    f7.plot(t, U)
    f8.plot(t, KE)
    f9.plot(t, U+KE)
    for atom in lattice2.atoms:
        if atom.visible:
            atom.pos =  atom.pos + .5*(atom.velocity+((atom.velocity)+atom.velocity*dt/t))*dt
    for spring in lattice2.springs:
        spring.axis = spring.points[1].pos - spring.points[0].pos
        L = mag(spring.axis)
        spring.axis = spring.axis.norm()
        spring.pos = spring.points[0].pos+0.15*spring.axis
        spring.length = L-atom.radius
        Force = spring.axis * (k*dt * (1-distance/L))
        
        if spring.points[0].visible:
            spring.points[0].momentum += Force
            spring.points[0].velocity = spring.points[0].momentum/m
        if spring.points[1].visible:
            spring.points[1].momentum -=  Force
            spring.points[1].velocity = spring.points[1].momentum/m
            
    
    v+=(-k)*dx*dt
    dx+=.5*(v*(t)+v*(t+dt))*dt
    t+=dt 
    KE=.5*m*v**2
    U=.5*k*dx**2

Comparing the results of the trapezoid to the other two methods (Forward and Backward Euler), we can see that it bares a much similar resemblence to the graph in the Backwards Euler method rather than the forward. The reason I choose the trapezoid equation was because of it resembles a mixture of the Forward and Backward as an average, to the naked eye. I figured this would make a strong comparison to the other two. Furthermore, we can take away that the later two methods assume that some unknown value is also being accounted for, and is constantly transfering some form of energy into the system. This would make sense, although over time in reality I feel as though this should decrease, there are constantly atoms being rearranged and restructured due to humanity, which means this may be a more realistic result, while the forward method seems to adopt a ideal world point of view. 

<h2>Damped Springs<h2>

In [17]:
from vpython import * 
scene = canvas()

<IPython.core.display.Javascript object>

In [18]:
k = 1
m = 1
distance = 1
r = .3
Length_spring = 1 - 1.8*.3
Volume_spring = pi*(.15)**2*Length_spring
dt = .1
t = .1
U = 0
KE = 0
spring_max = 0
v = 0
a=0
dx = Length_spring
critical_c = -2*(k)**.5
c = .5*critical_c

In [19]:
class Crystal:
        
    def __init__(self, radius, spacing, momentumRange):
        self.atoms = []
        self.springs = []
        
        for z in range(-1,4,1):
            for y in range(-1,4,1):
                for x in range(-1,4,1):
                    atom = sphere(pos=vector(x,y,z)*spacing, radius = radius, color = color.blue)
                    #only apply velocity to the visible atoms, the remaining atoms will keep the moving atoms in the intended place
                    if 0 <= x < 3 and 0 <= y < 3 and 0 <= z < 3:
                        p = vec.random()
                        atom.momentum = momentumRange*p
                        #for m = 1, p = v
                        atom.velocity = atom.momentum/m
                    else:#outer most layer is invisible
                        atom.visible = False
                        atom.velocity = vec(0,0,0)
                        atom.momentum = vec(0,0,0)
                    atom.index = len(self.atoms)
                    self.atoms.append(atom)
                    
        for atom in self.atoms:
            #determine if the adjacent atom should be invisible or visible
            if atom.visible:
                if atom.pos.x == 0 or atom.pos.x == 2:
                    self.place_spring("x", atom.pos.x, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+1], True)

                if atom.pos.y == 0 or atom.pos.y == 2:
                    self.place_spring("y", atom.pos.y, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)], True)
                    
                if atom.pos.z == 0 or atom.pos.z == 2:
                    self.place_spring("z", atom.pos.z, atom)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
    
    # Create a grid of springs linking each atom to the adjacent atoms
    # in each dimension, or to invisible motionless atoms
    def make_spring(self, start, end, visible):
        spring = helix(pos = start.pos, axis = end.pos-start.pos, visible = visible, points = [start, end], color = color.white)
        #spring characteristics for modeling
        spring.thickness = 0.05
        spring.radius = 0.15
        spring.length = distance
        self.springs.append(spring)
        
    def place_spring(self, c, n, atom):
        if(c=="x" and n==0):
            self.make_spring(self.atoms[atom.index-1], atom, False)
            self.make_spring(atom, self.atoms[atom.index+1], True)
        if(c=="x" and n==2):
            self.make_spring(atom, self.atoms[atom.index+1], False)
        if(c=="y" and n==0):
            self.make_spring(self.atoms[atom.index-(5)], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)], True)
        if(c=="y" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)], False)
        if(c=="z" and n==0):
            self.make_spring(self.atoms[atom.index-(5)**2], atom, False)
            self.make_spring(atom, self.atoms[atom.index+(5)**2], True)
        if(c=="z" and n==2):
            self.make_spring(atom, self.atoms[atom.index+(5)**2], False)
        


In [20]:
lattice2 = Crystal(r, distance, 0.1*distance*sqrt(k/m))

In [21]:
while t<10:
    rate(20)
    f10.plot(t, U)
    f11.plot(t, KE)
    f12.plot(t, U+KE)
    for atom in lattice2.atoms:
        if atom.visible:
            atom.pos =  atom.pos + atom.velocity*dt + .5*atom.velocity*dt**2
    for spring in lattice2.springs:
        spring.axis = spring.points[1].pos - spring.points[0].pos
        L = mag(spring.axis)
        spring.axis = spring.axis.norm()
        spring.pos = spring.points[0].pos+0.15*spring.axis
        spring.length = L-atom.radius
        Force = spring.axis * (k*dt * (1-distance/L))
        
        if spring.points[0].visible:
            spring.points[0].momentum += Force
            spring.points[0].velocity = spring.points[0].momentum/m
        if spring.points[1].visible:
            spring.points[1].momentum -=  Force
            spring.points[1].velocity = spring.points[1].momentum/m
    #where c is a constant determined by 2*sqrt(k*m) and where m=1        
    a=(-k*dx+c*v)/m
    v+=a*dt
    dx+=v*dt
    t+=dt 
    KE=.5*m*v**2
    U=.5*k*dx**2

The critical damping constant can be found by taking the sqrt of k x m and multiplying it by 2. Once this number is aquired, any number with a ratio between .3 and .7 can be considered a good dampening constant for constant/critical_constant. in order to do this i just took half the critical constant for a dampening constant of .5. using this we can see that the energy exponentially dissapates from the system due to the atoms using the energy they initially have to move the springs, transferring and releasing said energy. This would be a good example of atoms coming to rest after a person has walked directly next to them. However due to the constant motion of objects on the planet, the likelihood of the atoms coming to a full stop is almost zero.

<h1>References<h1>

https://www.youtube.com/watch?v=BIhS5atvK1o

https://eng.libretexts.org/Bookshelves/Electrical_Engineering/Signal_Processing_and_Modeling/Introduction_to_Linear_Time-Invariant_Dynamic_Systems_for_Students_of_Engineering_(Hallauer)/12%3A_Vibration_Modes_of_Undamped_Mechanical_Systems_with_Two_Degrees_of_Freedom/12.02%3A_Undamped_Two-Mass-Two-Spring_System

https://punchlistzero.com/damping-coefficient/#:~:text=A%20damping%20ratio%20of%201%20is%20ideal%20and,good%20damping%20have%20values%20between%200.3%20and%200.7.

https://www.glowscript.org/#/user/matterandinteractions/folder/matterandinteractions/program/04-ball-and-spring-model

https://math.stackexchange.com/questions/61638/whats-the-difference-between-explicit-and-implicit-runge-kutta-methods#:~:text=The%20Euler%20method%20is%20an%20explicit%20method%20in,consider%20both%2C%20when%20the%20explicit%20methods%20look%20simpler%3F

https://www.watelectronics.com/damping-ratio/#:~:text=Definition%3A%20The%20damping%20ratio%20is%20defined%20as%20the,For%20example%2C%20suspension%20of%20mass%20from%20a%20spring.

https://www.youtube.com/watch?v=_CDxdUgUNBA

https://www.youtube.com/watch?v=Cbaz95n2-fk